In [1]:
import math
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import WordNetLemmatizer
from gensim.models import *
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn import tree, ensemble
import random
import string
from nltk.stem.snowball import SnowballStemmer
from scipy.stats.stats import pearsonr
from sklearn import svm
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

%matplotlib inline

C:\Anaconda2\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [15]:
# read dataframes
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
X_train = pd.read_csv('sel_X_train.csv')
X_test = pd.read_csv('sel_X_test.csv')

In [16]:
# delete unused ids 
train.drop('Unnamed: 0', 1, inplace=True)
X_train.drop('Unnamed: 0', 1, inplace=True)
X_test.drop('Unnamed: 0', 1, inplace=True)

In [5]:
# create target value
y_train = train['relevance'].tolist()

#Models testing

For test each model apply cross_val_score with 10 folds.

In [17]:
# Linear model with l2 regularization
print cross_val_score(linear_model.Ridge(), X_train, y_train, cv = 10, scoring = 'neg_mean_absolute_error').mean()

-0.400616917194


In [18]:
# Linear model with l2 regularization
print cross_val_score(linear_model.Lasso(), X_train, y_train, cv = 10, scoring = 'neg_mean_absolute_error').mean()

# And all coef equals zero
model = linear_model.Lasso()
model.fit(X_train, y_train)
print model.coef_
# It's not good information about our features

-0.439455082754


In [23]:
# ensemble of decision trees
for est_num in range(30, 130, 30):
    for dep in range(4, 9, 1):
        print est_num, dep, cross_val_score(ensemble.RandomForestRegressor(n_estimators = est_num, max_depth=dep, random_state=0), 
                                            X_train, y_train, cv = 10, scoring = 'neg_mean_absolute_error').mean()

 30 4

 -0.392177182528
30 5

 -0.392283943338
30 6

 -0.391480924519
30 7

 -0.391309605325
30 8

...

 -0.392036209499
120 5

 -0.392029344773
120 6

 -0.391313143197
120 7

 -0.391212261501
120 8

 -0.390996604063


Increasing number of tree decreases standard deviation, but much number of trees will slow down our model.
Increasing max depth of trees leads to overfitting.

Let's take number of trees equals 30 and max depth equals 6.

In [24]:
# ensemble of decision trees
print cross_val_score(ensemble.RandomForestRegressor(n_estimators = 30, max_depth=6, random_state=0), 
                      X_train, y_train, cv=10, scoring = 'neg_mean_absolute_error').mean()

-0.391480924519


In [25]:
# ensemble of random forests with bagging
print cross_val_score(ensemble.BaggingRegressor(ensemble.RandomForestRegressor(n_estimators=30, max_depth=6, 
                                                                               random_state=0), 50, random_state=0), 
                      X_train, y_train, cv=10, scoring = 'neg_mean_absolute_error').mean()

-0.391419425963


In [26]:
# final calculation and create output file with prediction
random_forest = ensemble.RandomForestRegressor(n_estimators=30, max_depth=6, random_state=0)
model = ensemble.BaggingRegressor(random_forest, 50, random_state=0)

model.fit(X_train, y_train)

# calc mean_absolute_error for train set
res = 0
data2 = model.predict(X_train)
for i in range(len(data2)): # for linear models
    data2[i] = min(max(1.0, data2[i]), 3.0)
   
for i in range(len(y_train)):
    res += abs(y_train[i] - data2[i])

print res / len(y_train)
#

data = model.predict(X_test)
for i in range(len(data)): # for linear models
    data[i] = min(max(1.0, data[i]), 3.0)

output = pd.DataFrame({'id': test.id, 'relevance': data}).set_index('id')

output.to_csv('output.csv', sep=',')

0.388958385374
